# 第４ステージ　ニューラルネットワークを作る

## ステップ37 テンソルを扱う

### 37.1 要素ごとの計算

In [1]:
import numpy as np
import leopard.functions as F
from leopard import Variable

In [2]:
# 入力と出力が全てスカラ
x = Variable(np.array(1.0))
y = F.sin(x)
print(y)

variable(0.8414709848078965)


In [3]:
# 入力がテンソル -> 入力の各要素に適用
x = Variable(np.array([[1, 2, 3], [4, 5, 6]]))
y = F.sin(x)
print(y)

variable([[ 0.84147098  0.90929743  0.14112001]
          [-0.7568025  -0.95892427 -0.2794155 ]])


In [4]:
x = Variable(np.array([[1, 2, 3], [4, 5, 6]]))
c = Variable(np.array([[10, 20, 30], [40, 50, 60]]))
y = x + c
print(y)

variable([[11 22 33]
          [44 55 66]])


### 37.2 テンソルを使用したときのバックプロパゲーション
これまで「スカラ」を対象に微分を求めたが、「テンソル」でも正しく動作する。

In [5]:
x = Variable(np.array([[1, 2, 3], [4, 5, 6]]))
c = Variable(np.array([[10, 20, 30], [40, 50, 60]]))
y = x + c

y.backward(retain_grad=True) # retain_grad=Trueで微分を保持する
print(y.grad)
print(x.grad)
print(c.grad)

variable([[1 1 1]
          [1 1 1]])
variable([[1 1 1]
          [1 1 1]])
variable([[1 1 1]
          [1 1 1]])


### 37.3 省略

## ステップ38　形状を変える関数

### 38.1　reshape関数の実装
例えば、変数の形状を(2, 3)から(6,)へと変換する場合の逆伝播は、(6,)から(2, 3)へと変換する。

In [15]:
x = Variable(np.array([[1, 2, 3], [4, 5, 6]]))
y = F.reshape(x, (6,))
y.backward(retain_grad=True)
print(x.grad, x.grad.shape, y.shape)

variable([[1 1 1]
          [1 1 1]]) (2, 3) (6,)


### 38.2　Variableからreshapeを使う
現状のreshape関数をNumpyのそれに近づける。

In [7]:
x = np.random.rand(1, 2, 3)

y = x.reshape((2, 3)) # tuple
y = x.reshape([2, 3]) # list
y = x.reshape(2, 3) # そのまま

In [8]:
x = Variable(np.random.randn(1, 2, 3))
y = x.reshape((2, 3))
y = x.reshape(2, 3)

### 38.3　行列の転置
順伝播で(3, 2)を(2, 3)にしたら、逆伝播では、(2, 3)を(3, 2)にする。
つまり、reshapeと同様に順伝播の逆をすると良い。

In [9]:
x = Variable(np.array([[1, 2, 3], [4, 5, 6]]))
y = F.transpose(x)  # y = x.T
y.backward()
print(x.grad)

variable([[1 1 1]
          [1 1 1]])


In [10]:
x = Variable(np.random.rand(2, 3))
y = x.transpose()
print(y)
y = x.T
print(y)

variable([[0.84294614 0.62005639]
          [0.44507502 0.32517593]
          [0.37869763 0.2816437 ]])
variable([[0.84294614 0.62005639]
          [0.44507502 0.32517593]
          [0.37869763 0.2816437 ]])


### 38.4 【補足】実際のtranspose関数

In [13]:
A, B, C, D = 1, 2, 3, 4

x = Variable(np.random.rand(A, B, C, D))
print(x.shape)

y = x.transpose(1, 0, 3, 2)
y.backward()
print(x.grad.shape)

(1, 2, 3, 4)
(1, 2, 3, 4)


- memo<br>
    以下、`leopard/functions.py`のTranspose関数から抜粋
    ```Py
    class Transpose(Function):
        ...

        def backward(self, gy):
            if self.axes is None:
                return transpose(gy)

            axes_len = len(self.axes)
            inv_axes = tuple(np.argsort([ax % axes_len for ax in self.axes]))
            return transpose(gy, inv_axes)
    ```
    `inv_axes`の部分で、入力と同じになるように並び替えた次元のタプルが生成されている。<br>
    以下、並び替えの実験

In [19]:
A, B, C, D = 1, 2, 3, 4
axes = (D-1, A-1, B-1, C-1)
print(axes)

(3, 0, 1, 2)


- `np.argsort`は、引数を小さい順に並び替えた際の、元の引数のindexを返す。<br>
    上記の例で言うと、<br>
    `(index[0]: 3, index[1]: 0, index[2]: 1, index[3]: 2)`<br>
    ` -> (index[1]: 0, index[2]: 1, index[3]: 2, index[0]: 3)`

In [20]:
inv_axes = tuple(np.argsort(axes))
print(inv_axes)

(1, 2, 3, 0)


In [22]:
# 昇順に戻っている
[axes[i] for i in inv_axes]

[0, 1, 2, 3]